In [2]:
import pandas as pd
combined_dataset = pd.read_pickle('../combined_dataset_notes.pkl')

In [3]:

combined_dataset = combined_dataset.drop(['HADM_ID'], axis=1)
print(combined_dataset.shape)
combined_dataset.head()

(45526, 504)


,text,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
47366,"[chest, pa, lat, clip, clip, number, radiology...",7,0,35,462.0,0.0,24.0,26.0,1.0,104.0,...,0,1,0,0,0,0,0,0,0,1
47024,"[pm, liver, gallbladder, u, single, organ, cli...",5,1,60,462.0,-6.0,24.0,17.0,1.0,104.0,...,0,1,0,0,0,1,0,0,0,0
9046,"[chest, portable, ap, clip, clip, number, radi...",13,0,49,462.0,0.0,24.0,26.0,1.0,104.0,...,0,0,0,0,0,1,0,0,0,1
843,"[pm, chest, pa, lat, clip, clip, number, radio...",5,1,61,462.0,2.0,24.0,28.0,1.0,103.0,...,0,0,0,0,0,0,0,1,0,1
43466,"[pm, chest, portable, ap, different, physician...",5,0,54,462.0,0.0,23.0,25.0,1.0,106.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
only_text = combined_dataset[["text","G1","G2","G3","G4","G5","G6","G7","G8","G9","G10","G11","G12","G13","G14","G15","G16","G17","G18","G19","G20"]]
only_text.shape

(45526, 21)

In [5]:
!pip install --upgrade gensim
!pip install -U gensim

Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)


In [6]:

from gensim.models import Word2Vec

In [7]:
#pass text here..
#model = Word2Vec(combined_dataset.text,size=500, window=10, min_count=5, workers=56)#play around with param
# summarize the loaded model
#print(model)
# summarize vocabulary
#words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['sentence'])
# save model
#model.save('model5000-10-5.bin')
# load model
model = Word2Vec.load('model300-10-5.bin')
print(model)

Word2Vec(vocab=23960, size=300, alpha=0.025)


In [8]:
y = only_text.iloc[:,1:]
x = only_text.iloc[:,:1]

In [9]:
y.shape

(45526, 20)

In [10]:
from sklearn.model_selection import train_test_split
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(38697, 1) (6829, 1) (38697, 20) (6829, 20)


In [10]:
#voila

In [11]:
#avg_vec.shape

In [13]:
import numpy as np
WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss

#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=1000,
#                                                                     learning_rate=0.15))

#from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
import numpy as np
pipe1=Pipeline([("wordVectz",AverageEmbeddingVectorizer(WordVectorz)),
                ("multilabel",OneVsRestClassifier(LGBMClassifier(iterations=1000,learning_rate=0.15,n_jobs=-1))),
               ("multilabel2",OneVsRestClassifier(LGBMClassifier(iterations=1500,learning_rate=0.15,n_jobs=-1)))])
pipe1.fit(x_train.text,y_train)#run this and see but not here lets do this in a new nb
# clf_multilabel.fit(x_train.text,y_train)#have to now oopsitsk
#ok?run again from top right? haan go on wait what both lgbm... yea there is a feeling i have just wanna try.. k

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             iterations=1000,
                                             learning_rate=0.15, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=None, reg_alpha=0.0,
                                        

In [255]:
predicted1=pipe1.predict(x_test.text)

In [29]:
predicted1_prob_train = pipe1.predict_proba(x_train.text)
predicted1_prob_test = pipe1.predict_proba(x_test.text)

In [256]:
print(jaccard_score(y_test,predicted1,average='micro'))
print(jaccard_score(y_test,predicted1,average='macro'))
print(roc_auc_score(y_test,predicted1))
print(hamming_loss(y_test,predicted1))
'''
Cat w2v 500,10,5
0.6633380308436122
0.18253038512227265'''

0.5034983002608902
0.39044713301507433
0.6646507472338611
0.1839288329184361


'\nCat w2v 500,10,5\n0.6633380308436122\n0.18253038512227265'

In [89]:
predicted1.shape #earlier with 1500d & mincount=3 - 0.6573500885474709

(6829, 19)

In [44]:
#pipe1.fit(x_train.text,y_train)
predicted1_prob_testdf = pd.DataFrame(predicted1_prob_test)
predicted1_prob_traindf = pd.DataFrame(predicted1_prob_train)

In [45]:
predicted1_prob_test.shape

(6829, 20)

In [46]:
predicted1_prob_testdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.006364,0.014811,0.381461,0.244952,0.226117,0.133170,0.422299,0.114607,0.154242,0.135912,0.000083,0.029295,0.206063,0.008920,0.000142,0.144964,0.027488,0.984861,0.983045,0.347243
1,0.060448,0.020596,0.792334,0.469381,0.554800,0.337727,0.471109,0.146749,0.755750,0.457869,0.001473,0.103819,0.162318,0.017451,0.000144,0.527361,0.014730,0.352813,0.349358,0.289949
2,0.011747,0.047712,0.474600,0.111793,0.401464,0.651509,0.855404,0.151393,0.127526,0.281706,0.000056,0.029521,0.174612,0.006954,0.000131,0.310654,0.001712,0.515857,0.806501,0.454475
3,0.009529,0.030799,0.778939,0.136471,0.182365,0.104084,0.989231,0.141261,0.191664,0.090097,0.000048,0.020778,0.120590,0.015656,0.000347,0.082227,0.001190,0.168287,0.069491,0.452310
4,0.155675,0.350358,0.850952,0.722703,0.364650,0.714532,0.905861,0.852393,0.701567,0.771462,0.001481,0.397246,0.220130,0.035454,0.000101,0.672055,0.041518,0.754205,0.359311,0.675329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0.095784,0.137530,0.860846,0.712012,0.310655,0.738019,0.927311,0.315240,0.471653,0.871125,0.000228,0.494851,0.312031,0.008207,0.000102,0.528199,0.029597,0.859444,0.631167,0.771659
6825,0.070175,0.068985,0.846435,0.632877,0.799036,0.569135,0.780485,0.465946,0.804712,0.705267,0.001412,0.456766,0.196635,0.019320,0.000352,0.601936,0.041650,0.657041,0.343640,0.647979
6826,0.042937,0.040011,0.640910,0.506048,0.862476,0.538513,0.452046,0.583061,0.328257,0.358777,0.000726,0.209272,0.294012,0.011878,0.000791,0.636439,0.031056,0.493248,0.481291,0.412246
6827,0.105662,0.080924,0.846555,0.791480,0.317569,0.542417,0.959383,0.258478,0.417617,0.915276,0.000217,0.367771,0.196251,0.009987,0.000130,0.452880,0.026391,0.855679,0.562148,0.643258


x_train.text.to_pickle('./train-nlp.pkl')
x_test.text.to_pickle('./test-nlp.pkl')
y_train.to_pickle('./y_train-nlp.pkl')
y_test.to_pickle('./y_test-nlp.pkl')
predicted1_prob_testdf.to_pickle('./pred-test-xgb.pkl')
predicted1_prob_traindf.to_pickle('./pred-train-xgb.pkl')

In [174]:
from sklearn.externals import joblib
joblib.dump(pipe1, "w2v_lgbm.pkl")

['w2v_lgbm.pkl']

In [175]:
textlgbm =  joblib.load("w2v_lgbm.pkl")


In [151]:
valc = joblib.load("cat1-C.pkl")
textc = joblib.load("./w2v.pkl")
lgbm = joblib.load("./LGBM1-C.pkl")

KeyboardInterrupt: 

In [107]:
x_full = combined_dataset.iloc[:,:484]
y_full = combined_dataset.iloc[:,484:]

In [109]:
x_full.columns

Index([          'text', 'ADMISSION_DAYS',         'GENDER',            'AGE',
                  50801,            50802,            50803,            50804,
                  50805,            50806,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=484)

In [110]:
from sklearn.model_selection import train_test_split
seed=243 
x_train_full,x_test_full,y_train_full,y_test_full = train_test_split(x_full,y_full,test_size=0.15,random_state=seed)
x_test_full.iloc[:,1:484]

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
21738,10,1,83,462.0,5.0,24.0,32.0,1.0,104.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
47854,3,1,66,462.0,-2.0,22.0,22.0,1.0,105.0,1.10,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
3615,8,1,80,462.0,1.0,24.0,26.0,1.0,104.0,1.15,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
37273,5,1,46,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,1.0,1.044,4.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
12132,27,1,85,338.0,-1.0,24.0,25.0,1.0,100.0,1.14,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5542,16,1,82,584.0,0.0,24.0,26.0,1.0,104.0,1.12,...,1.0,1.044,0.2,1.0,7.0,3.0,0.0,0.0,135.5,43.0
57534,13,1,56,462.0,0.0,24.0,29.0,1.0,102.0,1.25,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
40358,12,1,36,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
24988,14,1,50,391.0,6.0,24.0,32.0,1.0,101.0,1.18,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0


In [259]:
y_pred_catnum = valc.predict(x_train_full.iloc[:,1:484])
y_pred_catext = textc.predict(x_train_full.text)

print(y_pred_catnum.shape,y_pred_catext.shape)

(38697, 20) (38697, 20)


In [279]:
y_pred_lg = lgbm.predict(x_train_full.iloc[:,1:484])
y_pred_lg_prob = lgbm.predict_proba(x_train_full.iloc[:,1:484])
print(y_pred_lg.shape)

(38697, 20)


In [278]:
y_pred_lgbm = textlgbm.predict(x_test_full.text)#need str stuff ? this is text only, we were using lgbm for str and cat for text
y_pred_lgbm_prob = textlgbm.predict_proba(x_test_full.text)

print(y_pred_lgbm_text.shape)

TypeError: 'int' object is not iterable

In [203]:
y_pred_lgbm_text = textlgbm.predict(x_test_full.text)
y_pred_lgbm_text_prob = textlgbm.predict_proba(x_test_full.text)

print(y_pred_lgbm_text.shape)

(6829, 20)


In [120]:
y_pred_catnum_prob = valc.predict_proba(x_test_full.iloc[:,1:484])
y_pred_catext_prob = textc.predict_proba(x_test_full.text)
print(y_pred_catnum.shape,y_pred_catext.shape)

(6829, 20) (6829, 20)


In [280]:
#what all is needed lol 
# i need lgbm for str and cat for text on train or test or both? both na?
y_pred_lg_prob_train = lgbm.predict_proba(x_train_full.iloc[:,1:484])
y_pred_lg_prob_test = lgbm.predict_proba(x_test_full.iloc[:,1:484])
y_pred_catext_prob_train = textc.predict_proba(x_train_full.text)
y_pred_catext_prob_test = textc.predict_proba(x_test_full.text)

In [292]:
y_comb =y_pred_lg_prob_test+np.subtract(1,y_pred_catext_prob)
#y_comb/=2
y_comb = y_comb.round()
y_comb = np.where(y_comb>1,1,0)#wait u got this wrong
#just try this, according to me text can give more info, and i would change w2v to 1000 
# 0.7052650131769738
# 0.1599868209108215

In [293]:
np.unique(y_comb)

array([0, 1])

In [294]:
# predicting on avg of both probs on test set
print(jaccard_score(y_test,y_comb,average='micro'))
print(jaccard_score(y_test,y_comb,average='macro'))
print(roc_auc_score(y_test,y_comb))
print(hamming_loss(y_test,y_comb))
#0.6646507472338611

0.07168119091098218
0.13843890622090185
0.561602477838084
0.29404012300483234


In [276]:
# predicting on train set
print(jaccard_score(y_train_full,y_comb,average='micro'))
print(jaccard_score(y_train_full,y_comb,average='macro'))
print(roc_auc_score(y_train_full,y_comb))
print(hamming_loss(y_train_full,y_comb))
#0.6646507472338611

0.6937055799183427
0.648084064314032
0.8135841276855682
0.10468640979920923


In [282]:
auc_sum = 0.7664439688584423/0.17394371656717575 + 0.8135841276855682/0.10468640979920923
#auc/ham worked...
y_comb = (y_pred_lg_prob_test*0.7664439688584423/0.17394371656717575 +y_pred_catext_prob_test*0.8135841276855682/0.10468640979920923)/auc_sum
y_comb = y_comb.round()

In [222]:
import numpy as np
def corr(a, b):
    counts = 0
  #first_df = pd.read_csv(first_file,index_col=0)
  #second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
    for i in range(a.shape[0]):  
      #prediction = a[:,i] #first_df.columns[0]
      # correlation
        #print("Finding correlation row : {}".format(i))
      #print("Column to be measured: {}".format(prediction))
        cor = np.corrcoef(a[i,:],b[i,:])
        #print("Pearson's correlation score: {}".format(cor))
        if(cor[0][1] < 0.7):
            counts+=1
      #print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
      #print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))
    return counts

In [223]:
l = [corr(y_pred_catnum,y_pred_lg)]
l# this was high thats y its working better ig
#what was high? like correlation was low for many inspite o
# 4717 cat v cat
# 4909 lgbm vs cat

[521]

In [ ]:
estimators=[ ('text', textc), ('val', valc)]
ensemble = OneVsRestClassifier(VotingClassifier(estimators, voting='soft'))
ensemble.fit(x_train_mm, y_train)

In [ ]:
def softmax(x):
    return 
    